<a href="https://colab.research.google.com/github/DedeBrahma/Deep-Learning-Yotube-Comment/blob/master/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Team =


1.   18917109 - Dede Brahma Arianto
2.   19917019 - Yopi Azani



# Load Dataset

In [0]:
import unicodedata
import re
import pandas as pd

In [0]:
dataset = pd.read_csv('/content/drive/My Drive/Kampus/Deep Learning Fundamental/Tugas_1 DL/ytb_comment.csv')
# dataset = dataset.drop(columns="Username")

In [0]:
dataset

,Username,Comment
0,Muhammad Irsyad,Ayo ayo jangan bosen2 saling mengingatkan...se...
1,Dion Damaris,Hargailah baik-baik kesehatan kita. Patuhi anj...
2,Woh Getoh,Update 22 Maret :-Malaysia 🇲🇾Total kes : 1306
3,Sugenghar Ianto,Bagi yang punya duit Borong semua masker boron...
4,ican Gofur,"Yang displin,yang tetap jaga kesehatan dan ber..."
...,...,...
765,firman adnan,Santuy badai pasti berlalau.. walau istriku la...
766,Mirza Ahmad,"Di luar jawa mungkin lebih banyak, apalagi Man..."
767,Lukia Stream,Dan detik ini Indonesia dapet ranking 1 Persen...
768,Raynaldi Eka Kesuma Putra,Jokowi Indonesia Harus Lockdown Sekarang Juga


# Text Cleaning

In [0]:
pip install sastrawi

     |████████████████████████████████| 215kB 2.5MB/s 


In [0]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import nltk

Masukkan stopwords_id.txt kedalam folder

`/root/nltk_data/corpora/stopwords`

kemudian rename menjadi stopwords_id (hilangkan extention txt)

In [0]:
def removeStopword(str):
    stop_words = set(stopwords.words('stopwords_id'))
    word_tokens = word_tokenize(str)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    return ' '.join(filtered_sentence)

def stemming(str):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    return stemmer.stem(str)

def cleaning(str):
    #remove non-ascii
    str = unicodedata.normalize('NFKD', str).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    #remove URLs
    str = re.sub(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))', '', str)
    #remove punctuations
    str = re.sub(r'[^\w]|_',' ',str)
    #remove digit from string
    str = re.sub("\S*\d\S*", " ", str).strip()
    #remove digit or numbers
    str = re.sub(r"\b\d+\b", " ", str)
    #to lowercase
    str = str.lower()
    #Remove additional white spaces
    str = re.sub('[\s]+', ' ', str)
    # print(str)   
    return str

def slangWord(str):
  # str = str.lower()
  # file = '/slang.dic'
  file = './slang.dic'
  fix2 = {}
  with open(file) as f:
    for line in f:
       (key, val) = line.split(':')
       fix2[key] = val.replace('\n',' ')
  tweet_t = nltk.word_tokenize(str)
  clean = " "
  for i in tweet_t:
    for slang, formal in fix2.items():
        if i == slang:
            i = formal
    if i.isalnum():
        clean = clean+i+' '
    else:
        clean = clean[:-1]+' '+i+' '
  str = clean
  return str

def preprocessing(str):    
    str = cleaning(str)
    str = slangWord(str)        
    str = removeStopword(str)
    str = stemming(str)
    
    return str

In [0]:
data_clean = []
data_text = dataset['Comment']
for item in data_text:
  tmp = preprocessing(item)
  data_clean.append(tmp)

dataset['Comment'] = data_clean

In [0]:
dataset

,Username,Comment
0,Muhammad Irsyad,ayo ayo moga allah swt lindung amin
1,Dion Damaris,harga sehat patuh anjur perintah sehat sakit
2,Woh Getoh,update maret malaysia total kes
3,Sugenghar Ianto,duit borong masker borong sanitizer borong bah...
4,ican Gofur,displin jaga sehat berfikir positiv tahan sola...
...,...,...
765,firman adnan,santuy badai lalau istri hamil moga aja ga
766,Mirza Ahmad,jawa manado jakarta gubernur anies baswedan pe...
767,Lukia Stream,detik indonesia dapet ranking persentase mati ...
768,Raynaldi Eka Kesuma Putra,jokowi indonesia lockdown


# Klasifikasi Sentiment

In [0]:
from textblob import TextBlob
def analize_sentiment(comment):
  # print(comment)
  try:
    analysis = TextBlob(str(comment))
    time.sleep(0.2)
    toEN = analysis.translate(to='en')
    time.sleep(0.2)
    if toEN.sentiment.polarity > 0:
      return('positif')
    elif toEN.sentiment.polarity == 0:
      return('netral')
    else:
      return('negatif')
  except:
    return ('netral')

In [0]:
# Tambahkan kolom label ke dalam dataframe:
dataset['label'] = np.array([analize_sentiment(tweet) for tweet in dataset['Comment'] ])

# Tampilkan data:
display(dataset)

,Username,Comment,label
0,Muhammad Irsyad,ayo ayo moga allah swt lindung amin,netral
1,Dion Damaris,harga sehat patuh anjur perintah sehat sakit,netral
2,Woh Getoh,update maret malaysia total kes,netral
3,Sugenghar Ianto,duit borong masker borong sanitizer borong bah...,netral
4,ican Gofur,displin jaga sehat berfikir positiv tahan sola...,netral
...,...,...,...
765,firman adnan,santuy badai lalau istri hamil moga aja ga,netral
766,Mirza Ahmad,jawa manado jakarta gubernur anies baswedan pe...,netral
767,Lukia Stream,detik indonesia dapet ranking persentase mati ...,netral
768,Raynaldi Eka Kesuma Putra,jokowi indonesia lockdown,netral


In [0]:
pos_tweets = [ tweet for index, tweet in enumerate(dataset['Comment']) if dataset['label'][index] == 'positif']
neu_tweets = [ tweet for index, tweet in enumerate(dataset['Comment']) if dataset['label'][index] == 'netral']
neg_tweets = [ tweet for index, tweet in enumerate(dataset['Comment']) if dataset['label'][index] == 'negatif']

In [0]:
total_pos = len(pos_tweets)*100/len(dataset['Comment'])
total_net = len(neu_tweets)*100/len(dataset['Comment'])
total_neg = len(neg_tweets)*100/len(dataset['Comment'])

print("Total Positif: ", total_pos)
print("Total Netral: ", total_net)
print("Total Negatif: ", total_neg)

Total Positif:  4.675324675324675
Total Netral:  92.72727272727273
Total Negatif:  2.5974025974025974


In [0]:
dataset = dataset.drop(columns="Username")
dataset

In [0]:
dataset.to_csv('/content/drive/My Drive/Kampus/Deep Learning Fundamental/Tugas_1 DL/clean_data.csv', index = False)